In [1]:
!pip install torch transformers sentence_transformers spacy>=3.5 gradio
!python -m spacy download fr_core_news_md

2023-02-22 14:02:03.896833: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 14:02:06.207764: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-22 14:02:06.207886: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-22 14:02:06.207909: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

# import all the necessary dependent files

In [2]:
import requests
from pathlib import Path

if Path('summarizer.py').is_file():
    print('already exists')
else:
    print('not existed yet')
    request = requests.get("https://huggingface.co/spaces/Carlosito16/HXM-summarization/raw/main/app.py")
    with open('summarizer.py', "wb") as f:
        f.write(request.content)


if Path('helper_function.py').is_file():
    print('already exists')
else:
    print('not existed yet')
    request = requests.get("https://huggingface.co/spaces/Carlosito16/HXM-summarization/raw/main/helper_function.py")
    with open('helper_function.py', "wb") as f:
        f.write(request.content)


if Path('french_stopword.txt').is_file():
    print('already exists')
else:
    print('not existed yet')
    request = requests.get("https://raw.githubusercontent.com/stopwords-iso/stopwords-fr/master/stopwords-fr.txt")
    with open('french_stopword.txt', "wb") as f:
        f.write(request.content)

not existed yet
not existed yet
not existed yet


In [3]:
import torch
from transformers import RobertaTokenizerFast, EncoderDecoderModel, AutoTokenizer, AutoModelForSeq2SeqLM

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

import spacy
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

import requests
from pathlib import Path


In [4]:
class keyWordExtractor():
    
    def __init__(self, 
                 article_text,
                 similarity_model,
                 n_gram = 1,
                 top_n = 3,
                 french_stopwords = None,
                 ner= None,
                 ):
        self.article_text = article_text
        self.french_stopwords = french_stopwords
        self.candidates = self.count_vectorizer(n_gram)
        self.noun_candidates, self.proper_noun_candidates = self.slice_only_noun_token(ner, self.candidates)
        self.top_n_keywords = self.top_n_extractor(similarity_model, top_n)
    
    def count_vectorizer(self, n_gram):
        n_gram_range = (n_gram, n_gram)
        # Extract candidate words/phrases
        count = CountVectorizer(ngram_range=n_gram_range,
                        stop_words = self.french_stopwords).fit([self.article_text]) #Main change
        candidates = count.get_feature_names_out()

        return candidates

    def slice_only_noun_token(self, ner, token_list):
        """
        Given the tokenized list, this function returns only the "NOUN" token
            Args:
                ner (spacy): The NER class to detect the `token.pos_`
                token_list (list): List of token from the full article
            Returns:
                slice_list (list): List of token containing only "NOUN" part of speech
        """

        noun_slice_list = []
        proper_noun_slice_list = []
        for word_idx in range(len(token_list)):
            doc = ner(token_list[word_idx])

            for token in doc:
                if token.pos_ == 'NOUN':
                    noun_slice_list.append(token.text)
                elif token.pos_ == 'PROPN':
                    proper_noun_slice_list.append(token.text)

        return noun_slice_list, proper_noun_slice_list

    def top_n_extractor(self, model, top_n):
        doc_embedding = model.encode([self.article_text])
        candidate_embeddings = model.encode(self.noun_candidates)
        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        keywords = [self.noun_candidates[index] for index in distances.argsort()[0][-top_n:]]

        return keywords

def clear_input():
    return ("", "")

       
def camembert_generate_summary(article_text):
   inputs = cmb_tokenizer([article_text], padding="max_length", truncation=True,
                          max_length=50,
                          return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = cmb_model.generate(input_ids, attention_mask=attention_mask, max_length = 50)
   return cmb_tokenizer.decode(output[0], skip_special_tokens=True)

    
def t5_generate_summary(article_text):
    input_ids = t5_tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512)["input_ids"]
    
    output_ids = t5_model.generate(
        input_ids=input_ids,
        max_length=84,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]
    
    output = t5_tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
    )

    return output

    
def summarizer(dropdown_model, article_text):
    """
    Ruturs a summarized version from the full article based on the selected pretrained-model
    """

    if dropdown_model == 'camembert':
        summary = camembert_generate_summary(article_text)

    elif dropdown_model == 'T5':
        summary = t5_generate_summary(article_text)

    return summary
    
def extract_top_3(article):
    nlp = spacy.load("fr_core_news_md")
    # model = SentenceTransformer("dangvantuan/sentence-camembert-large") #

    a= keyWordExtractor(article,
                        n_gram = 1, 
                        top_n = 3,
                        ner = nlp,
                        similarity_model = model)
    keyword = ", ".join(a.top_n_keywords) #to return ['a' , 'b'] >> "a, b"
    proper_nonuns = ", ".join(a.proper_noun_candidates)

    return keyword, proper_nonuns


def runall(dropdown_model, article_text):
    summary = summarizer(dropdown_model, article_text)
    keywords, proper_n = extract_top_3(article_text)

    return summary, keywords, proper_n

---

# Let's stall calling the model

In [5]:
#set the device agnostics code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
test_article ="""\"Un nuage de fumée juste après l’explosion, le 1er juin 2019. Une déflagration dans une importante usine d’explosifs du centre de la Russie a fait au moins 79 blessés samedi 1er juin. L’explosion a eu lieu dans l’usine Kristall à Dzerzhinsk, une ville située à environ 400 kilomètres à l’est de Moscou, dans la région de Nijni-Novgorod. « Il y a eu une explosion technique dans l’un des ateliers, suivie d’un incendie qui s’est propagé sur une centaine de mètres carrés », a expliqué un porte-parole des services d’urgence. Des images circulant sur les réseaux sociaux montraient un énorme nuage de fumée après l’explosion. Cinq bâtiments de l’usine et près de 180 bâtiments résidentiels ont été endommagés par l’explosion, selon les autorités municipales. Une enquête pour de potentielles violations des normes de sécurité a été ouverte. Fragments de shrapnel Les blessés ont été soignés après avoir été atteints par des fragments issus de l’explosion, a précisé une porte-parole des autorités sanitaires citée par Interfax. « Nous parlons de blessures par shrapnel d’une gravité moyenne et modérée », a-t-elle précisé. Selon des représentants de Kristall, cinq personnes travaillaient dans la zone où s’est produite l’explosion. Elles ont pu être évacuées en sécurité. Les pompiers locaux ont rapporté n’avoir aucune information sur des personnes qui se trouveraient encore dans l’usine."""


cmb_ckpt = 'mrm8488/camembert2camembert_shared-finetuned-french-summarization'
cmb_tokenizer = RobertaTokenizerFast.from_pretrained(cmb_ckpt)
cmb_model = EncoderDecoderModel.from_pretrained(cmb_ckpt).to(device)


The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


In [6]:
print(device)

cuda


In [7]:
print(summarizer("camembert", test_article))

Un nuage de fumée juste après l’explosion, le 1er juin 2018, d’une explosion dans une usine d’explosifs russe a fait au moins 79 blessés.


## Also downlaod the spaCy

In [8]:
nlp = spacy.load("fr_core_news_md")
model =  SentenceTransformer("dangvantuan/sentence-camembert-large")

print(extract_top_3(test_article))

('explosifs, explosion, déflagration', 'moscou, novgorod, russie')


In [9]:
summary, keywords, proper_n = runall("camembert",  test_article)

In [10]:
proper_n

'moscou, novgorod, russie'

# Download that manuallly labeled we have aorund 270 rows

In [11]:
df = pd.read_csv("/content/20230220_selected_df.csv")
df.rename(columns= {"Unnamed: 0": "new_index"} , inplace=True)
df.head()

,new_index,original_index,ratings,reviewer,titles,reviews,english,dates,replies,reply_dates,...,Drive,Delivery,After Sales,Carrefour,Other brand,reimbursement,clean_BE,clean_PD,clean_DM,clean_AS
0,0,1945.0,1.0,All,Aucun SAV à éviter,"Nous avons commandé et reçu une tablette, à u...","We ordered and received a tablet, at a very go...",2014-11-01T11:09:09.000Z,NaN,NaN,...,False,False,True,False,True,False,1,1,0,1
1,1,826.0,1.0,All,impossible de se connecter er de créer…,impossible de se connecter er de créer un compte,Unable to login and create an account,2021-04-18T12:53:31.000Z,NaN,NaN,...,False,False,False,False,True,False,1,0,0,1
2,2,454.0,1.0,All,Mépris et arnaque,Mépris total pour le client et arnaque en mass...,NaN,2021-12-08T16:58:00.000Z,NaN,NaN,...,False,True,True,True,False,False,0,0,1,1
3,3,457.0,1.0,All,Carrefour traiteur à fuir,Mon expérience concerne Carrefour traiteur: PL...,NaN,2021-12-08T12:31:55.000Z,NaN,NaN,...,True,False,True,True,False,False,0,0,1,1
4,4,682.0,1.0,All,Zéro pointé,Zero pointé car opération mal conçue. Carref...,NaN,2021-05-02T12:04:10.000Z,NaN,NaN,...,False,False,False,False,False,False,1,0,0,0


In [21]:
from tqdm.auto import tqdm
import json

## Read the ongoing labeling if have

In [23]:
with open('/content/summary.json', 'r') as f:
  summary_dict = json.load(f)

In [29]:
summary_dict["0"]['top_3']

['courriels,', 'réponse,', 'patientons']

In [35]:
def save_json(dictname):
    # create json object from dictionary
    json_file = json.dumps(dictname , indent=4)

    # open file for writing, "w" 
    f = open("summary.json","w")

    # write json object to file
    f.write(json_file)

    # close file
    f.close()


# RUN THIS TO START SUMMARIZATION AND EXTRACTION AND KEEP IN JSON

In [34]:
summary_dict = dict()

In [40]:
for i in tqdm(range(34,50)):
    data = df.iloc[i]
    print(data.new_index) 
    if data.new_index in summary_dict: #check first if we already do this row to faster the process
        pass
    else:
        int_index = int(data.new_index) #need to transform this to string, otherwise the json.load does not allow the numpy.
        summary_dict[int_index] = {} #create the new nested dict with new_index as key
        summary, top_3, proper_n = runall("camembert", data.combined_reviews) #use the columns 'combined_reviews' colum
        summary_dict[int_index]['original'] = data.combined_reviews
        summary_dict[int_index]['summary'] = summary
        summary_dict[int_index]['top_3'] = top_3.split() #split to tranform strings into list
        summary_dict[int_index]['proper_n'] = proper_n.split()


save_json(summary_dict)

  0%|          | 0/16 [00:00<?, ?it/s]

34
35


RuntimeError: ignored

In [22]:
save_json(summary_dict)

In [33]:
summary_dict[5]

{'original': "Service de livraison et ou de Drive… Service de livraison et ou de Drive pratique mais qui devient laborieux.Trop de produits manquants facturés aussi bien en livraison à domicile qu'avec le Drive. On court après les sollicitations de remboursements successifs car les demandes ne sont pas traitées. Et quand elles le sont, les virements ne sont pas effectués ou bien ne correspondent pas au montant payé. La cerise sur le gâteau: un Drive non remis car indisponible le jour J débité quand même. Alors que j'ai pris la peine de prévenir le service client de cette déconvenue où l'on m'a assurée du non débit de ma carte bancaire me voilà encore contrainte de faire une demande de remboursement. Dorénavant par gain de temps j'irai faire mes courses moi même.  Le service est pitoyable (indépendamment du contexte actuel) et les démarches chronophages. Un paradoxe compte tenu de toutes les enquêtes de satisfaction reçues!!! Dommage",
 'summary': 'Service de livraison et ou de Drive...

In [ ]:
33

In [38]:
save_json(summary_dict)